In [0]:
#in the begening of the assignment import all the related library fot this assignment
# Import PySpark SQL functions
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.functions import sum, col

In [0]:
#/FileStore/tables/payment_table.csv
#uploads all the related table in the database (/FileStore/tables/payment-2.csv)

spark = SparkSession.builder.appName("paymentA2").getOrCreate()
payment_df = spark.read.option("header", "true").csv("/FileStore/tables/payment_table.csv")
payment_df.printSchema()

root
 |-- payment_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- staff_id: string (nullable = true)
 |-- rental_id: string (nullable = true)
 |-- amount: string (nullable = true)
 |-- payment_date: string (nullable = true)



In [0]:
#/FileStore/tables/rental_table-1.csv
spark = SparkSession.builder.appName("rentalA2").getOrCreate()
rental_df = spark.read.option("header", "true").csv("/FileStore/tables/rental_table-1.csv")
rental_df.printSchema()

root
 |-- rental_id: string (nullable = true)
 |-- rental_date: string (nullable = true)
 |-- inventory_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- return_date: string (nullable = true)
 |-- staff_id: string (nullable = true)
 |-- last_update: string (nullable = true)



In [0]:
display(payment_df.limit(5))
display(rental_df.limit(5))

payment_id customer_id staff_id rental_id amount payment_date 17503 341 2 1520 7.99 25:47.0 17504 341 1 1778 1.99 23:15.0 17505 341 1 1849 7.99 41:46.0 17506 341 2 2829 2.99 39:57.0 17507 341 2 3130 7.99 31:49.0

rental_id rental_date inventory_id customer_id return_date staff_id last_update 2 24/05/2005 22:54 1525 459 28/05/2005 19:40 1 16/02/2006 02:30 3 24/05/2005 23:03 1711 408 01/06/2005 22:12 1 16/02/2006 02:30 4 24/05/2005 23:04 2452 333 03/06/2005 01:43 2 16/02/2006 02:30 5 24/05/2005 23:05 2079 222 02/06/2005 04:33 1 16/02/2006 02:30 6 24/05/2005 23:08 2792 549 27/05/2005 01:32 1 16/02/2006 02:30

In [0]:
# Get the total number of rentals
total_rentals = rental_df.select("rental_id").distinct().count()

# Display the total number of rentals
print(f"Total number of rentals: {total_rentals}")

Total number of rentals: 16044


In [0]:


# Convert date strings to timestamp type
rental_df = rental_df.withColumn("rental_date", to_timestamp(col("rental_date"), "dd/MM/yyyy HH:mm"))
rental_df = rental_df.withColumn("return_date", to_timestamp(col("return_date"), "dd/MM/yyyy HH:mm"))
rental_df = rental_df.withColumn("last_update", to_timestamp(col("last_update"), "dd/MM/yyyy HH:mm"))

# Extract the year and month from the rental_date
rental_df = rental_df.withColumn("year", year(col("rental_date")))
rental_df = rental_df.withColumn("month", month(col("rental_date")))

# Group by year and month and count the number of rentals
rentals_per_month = rental_df.groupBy("year", "month").agg(count("rental_id").alias("rental_count"))

# Show the results
rentals_per_month.orderBy("year", "month").show()

+----+-----+------------+
|year|month|rental_count|
+----+-----+------------+
|2005|    5|        1156|
|2005|    6|        2311|
|2005|    7|        6709|
|2005|    8|        5686|
|2006|    2|         182|
+----+-----+------------+



In [0]:
display(rental_df.limit(15))

rental_id rental_date inventory_id customer_id return_date staff_id last_update year month 2 2005-05-24T22:54:00.000+0000 1525 459 2005-05-28T19:40:00.000+0000 1 2006-02-16T02:30:00.000+0000 2005 5 3 2005-05-24T23:03:00.000+0000 1711 408 2005-06-01T22:12:00.000+0000 1 2006-02-16T02:30:00.000+0000 2005 5 4 2005-05-24T23:04:00.000+0000 2452 333 2005-06-03T01:43:00.000+0000 2 2006-02-16T02:30:00.000+0000 2005 5 5 2005-05-24T23:05:00.000+0000 2079 222 2005-06-02T04:33:00.000+0000 1 2006-02-16T02:30:00.000+0000 2005 5 6 2005-05-24T23:08:00.000+0000 2792 549 2005-05-27T01:32:00.000+0000 1 2006-02-16T02:30:00.000+0000 2005 5 7 2005-05-24T23:11:00.000+0000 3995 269 2005-05-29T20:34:00.000+0000 2 2006-02-16T02:30:00.000+0000 2005 5 8 2005-05-24T23:31:00.000+0000 2346 239 2005-05-27T23:33:00.000+0000 2 2006-02-16T02:30:00.000+0000 2005 5 9 2005-05-25T00:00:00.000+0000 2580 126 2005-05-28T00:22:00.000+0000 1 2006-02-16T02:30:00.000+0000 2005 5 10 2005-05-25T00:02:00.000+0000 1824 399 2005-05-31T22:44:00.000+0000 2 2006-02-16T02:30:00.000+0000 2005 5 11 2005-05-25T00:09:00.000+0000 4443 142 2005-06-02T20:56:00.000+0000 2 2006-02-16T02:30:00.000+0000 2005 5 12 2005-05-25T00:19:00.000+0000 1584 261 2005-05-30T05:44:00.000+0000 2 2006-02-16T02:30:00.000+0000 2005 5 13 2005-05-25T00:22:00.000+0000 2294 334 2005-05-30T04:28:00.000+0000 1 2006-02-16T02:30:00.000+0000 2005 5 14 2005-05-25T00:31:00.000+0000 2701 446 2005-05-26T02:56:00.000+0000 1 2006-02-16T02:30:00.000+0000 2005 5 15 2005-05-25T00:39:00.000+0000 3049 319 2005-06-03T03:30:00.000+0000 1 2006-02-16T02:30:00.000+0000 2005 5 16 2005-05-25T00:43:00.000+0000 389 316 2005-05-26T04:42:00.000+0000 2 2006-02-16T02:30:00.000+0000 2005 5

In [0]:
# Convert the `amount` column to a numeric type
payment_df = payment_df.withColumn("amount", col("amount").cast("float"))

# Calculate the total sum of all payments
total_sum = payment_df.agg(sum("amount").alias("total_sum")).collect()[0]["total_sum"]

# Show the total sum of all payments
print(f"Total Sum of All Payments: {total_sum}")

Total Sum of All Payments: 61312.03846335411


In [0]:
# Aggregate payments by customer
aggregated_payments = payment_df.groupBy("customer_id").agg(sum("amount").alias("total_payments"))

# Find the top 5 customers by total payments
top_customers = aggregated_payments.orderBy(col("total_payments").desc()).limit(5)

# Show the top 5 customers
top_customers.show()

+-----------+------------------+
|customer_id|    total_payments|
+-----------+------------------+
|        148|211.54999470710754|
|        526|208.57999467849731|
|        178|194.60999464988708|
|        137|191.61999487876892|
|        144|189.59999537467957|
+-----------+------------------+



In [0]:
# Convert the `amount` column to a numeric type
payment_df = payment_df.withColumn("amount", col("amount").cast("float"))

# Aggregate the total payments collected by each staff member
total_payments_by_staff = payment_df.groupBy("staff_id").agg(sum("amount").alias("total_payments"))

# Show the results
total_payments_by_staff.orderBy(col("total_payments").desc()).show()

+--------+------------------+
|staff_id|    total_payments|
+--------+------------------+
|       2| 31059.91921567917|
|       1|30252.119247674942|
+--------+------------------+



In [0]:
# Example data for staff_df
staff_data = [
    ("1", "Prabin"),
    ("2", "Sharma")
]
staff_columns = ["staff_id", "staff_name"]
staff_df = spark.createDataFrame(staff_data, staff_columns)

# Convert the `amount` column to a numeric type in payment_df
payment_df = payment_df.withColumn("amount", col("amount").cast("float"))

# Join payment_df with staff_df on staff_id
joined_df = payment_df.join(staff_df, on="staff_id")

# Aggregate the total payments collected by each staff member
total_payments_by_staff = joined_df.groupBy("staff_id", "staff_name").agg(sum("amount").alias("total_payments"))

# Show the results
total_payments_by_staff.orderBy(col("total_payments").desc()).show()

+--------+----------+------------------+
|staff_id|staff_name|    total_payments|
+--------+----------+------------------+
|       2|    Sharma| 31059.91921567917|
|       1|    Prabin|30252.119247674942|
+--------+----------+------------------+

